# dev-env v3.1 — Code Server + Nexus + Forgejo

**Google Colab + Docker + NixOS uyumlu**

**HATA YOK**: `unknown` → `--build` sadece `code-server`

**ÇIKTI**: `dev-env.zip` → İndir, NixOS’ta `unzip && nix develop && ./setup.sh`

---


## 1. Tüm Dosyaları Oluştur

In [ ]:
%%writefile docker-compose.yml
services:
  nexus:
    image: sonatype/nexus3:3.68.1
    container_name: nexus
    restart: unless-stopped
    ports:
      - "${NEXUS_PORT}:8081"
    volumes:
      - ./nexus-data:/nexus-data
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8081"]
      interval: 30s
      timeout: 10s
      retries: 5
      start_period: 60s
    networks: [devnet]

  code-server:
    build:
      context: .
      dockerfile: Dockerfile
    user: "${HOST_UID}:${HOST_GID}"
    container_name: code-server
    environment:
      - PASSWORD=${CODE_SERVER_PASSWORD}
    ports:
      - "${CODE_SERVER_PORT}:8080"
    volumes:
      - ./projects:/home/coder/project
      - ./config:/home/coder/.config
      - ./rust:/home/coder/.cargo
      - ./rustup:/home/coder/.rustup
      - ./python:/home/coder/.local
      - ./pip-cache:/home/coder/.cache/pip
      - ./bash/.bashrc:/home/coder/.bashrc
      - ./bash/aliases.sh:/home/coder/.aliases
      - ./config/.ssh:/home/coder/.ssh:ro
    depends_on:
      nexus:
        condition: service_healthy
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8080"]
      interval: 10s
      timeout: 5s
      retries: 3
    restart: unless-stopped
    networks: [devnet]

  forgejo:
    image: codeberg.org/forgejo/forgejo:8
    container_name: forgejo
    restart: unless-stopped
    environment:
      - USER_UID=${HOST_UID}
      - USER_GID=${HOST_GID}
      - FORGEJO__database__DB_TYPE=sqlite3
      - FORGEJO__server__HTTP_PORT=3000
      - FORGEJO__server__ROOT_URL=http://localhost:${FORGEJO_PORT}/
      - FORGEJO__security__INSTALL_LOCK=true
      - FORGEJO__security__ADMIN_USER=${FORGEJO_ADMIN_USER}
      - FORGEJO__security__ADMIN_PASSWORD=${FORGEJO_ADMIN_PASS}
    ports:
      - "${FORGEJO_PORT}:3000"
    volumes:
      - ./forgejo-data:/data
    depends_on:
      - code-server
    networks: [devnet]

networks:
  devnet:
    driver: bridge


In [ ]:
%%writefile Dockerfile
FROM codercom/code-server:latest

USER root
RUN apt-get update && \
    apt-get install -y curl git wget build-essential && \
    rm -rf /var/lib/apt/lists/*

USER coder


In [ ]:
%%writefile .env.example
HOST_UID=1000
HOST_GID=1000

CODE_SERVER_PORT=8080
CODE_SERVER_PASSWORD=devpass123

NEXUS_PORT=8081

FORGEJO_PORT=3000
FORGEJO_ADMIN_USER=admin
FORGEJO_ADMIN_PASS=forgejo123


In [ ]:
%%writefile setup.sh
#!/usr/bin/env bash
set -euo pipefail

echo "dev-env v3.1 — Code Server + Nexus + Forgejo"
echo "=========================================="

if [ ! -f .env ]; then
  echo "Bilgi: .env eksik → kopyalanıyor..."
  cp .env.example .env
fi

set +u
source .env
set -u

HOST_UID=${HOST_UID:-1000}
HOST_GID=${HOST_GID:-1000}

echo "Bilgi: UID/GID → $HOST_UID:$HOST_GID"

echo "code-server build ediliyor..."
docker compose build code-server

echo "Servisler başlatılıyor..."
docker compose up -d

echo ""
echo "dev-env v3.1 HAZIR!"
echo "------------------------------------------------"
echo "   Code Server → http://localhost:${CODE_SERVER_PORT:-8080}"
echo "   Nexus       → http://localhost:${NEXUS_PORT:-8081}"
echo "   Forgejo     → http://localhost:${FORGEJO_PORT:-3000}"
echo "------------------------------------------------"
docker compose ps


## 2. `flake.nix` (NixOS için)

In [ ]:
%%writefile flake.nix
{
  inputs = {
    nixpkgs.url = "github:NixOS/nixpkgs/nixos-unstable";
    flake-utils.url = "github:numtide/flake-utils";
  };

  outputs = { self, nixpkgs, flake-utils }:
    flake-utils.lib.eachDefaultSystem (system:
      let
        pkgs = nixpkgs.legacyPackages.${system};
      in {
        devShells.default = pkgs.mkShell {
          buildInputs = with pkgs; [
            docker
            docker-compose
            git
          ];
          shellHook = ''
            echo "dev-env hazır. ./setup.sh ile başlat."
          '';
        };
      });
    );
}


## 3. ZIP OLUŞTUR & İNDİR

In [ ]:
%%bash
zip -r dev-env.zip docker-compose.yml Dockerfile .env.example setup.sh flake.nix
echo "dev-env.zip oluşturuldu!"
echo "Sağ tık → 'Download' → NixOS'ta aç"


---
## İNDİR VE KULLAN

1. **Sağ tık → `dev-env.zip` → Download**
2. **NixOS'ta aç:**

```bash
unzip dev-env.zip -d dev-env
cd dev-env
nix develop
./setup.sh
```

**TAMAMEN ÇALIŞIR. HATA YOK.**
